<a href="https://colab.research.google.com/github/orhod/Basic-MineSweeper/blob/main/Logic/CloudDBTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required Python packages
%pip install -q firebase
%pip install -q importnb

from firebase import firebase
from datetime import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
DBLink = "https://couldproject-a621d-default-rtdb.europe-west1.firebasedatabase.app/"

In [ ]:
FBconn = firebase.FirebaseApplication(DBLink, None)

def insert_to_db_index(results, page_count):
      FBconn.put('/','terms', results)
      stats = FBconn.get('/', 'indexStats') or {}
      stats["word_count"] = len(results)
      stats["page_count"] = page_count
      stats["last_indexed"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      FBconn.put('/','indexStats', stats)
      return stats["last_indexed"]

def get_from_db(path):
      results = FBconn.get('/',path)
      return results

def insert_to_db_sensor(location, data):
      FBconn.put('/', f'/Data/{location}', data)
      return

def init_stats_if_needed():
    stats = FBconn.get('/', 'indexStats')
    if not stats:
        # Initialize with default values
        default_stats = {
            "word_count": 0,
            "page_count": 0,
            "last_indexed": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "search_counts": {"placeholder": 0}  # Start with a placeholder entry
        }
        FBconn.put('/', 'indexStats' , default_stats)

def insert_to_db_stats(stats):
    FBconn.put('/', 'indexStats' , stats)
    return

def add_task(task):
    # Get current tasks to generate a simple unique ID
    tasks = FBconn.get('/', 'Tasks') or {}
    # Generate a simple unique ID based on current timestamp
    task_id = f"task_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
    # Add the task with a unique key
    FBconn.put('/Tasks', task_id, task)
    return task_id

def insert_to_db_task(task):
    FBconn.put('/', 'Tasks' , task)
    return

def delete_task(task_id):
    """Delete a task by its ID"""
    try:
        FBconn.delete('/Tasks', task_id)
        return True
    except Exception as e:
        print(f"Error deleting task: {e}")
        return False

def delete_task_by_name(task_name):
    """Delete a task by its name - for backward compatibility"""
    tasks = FBconn.get('/', 'Tasks') or {}
    task_id_to_delete = None

    for task_id, task_data in tasks.items():
        if isinstance(task_data, dict) and task_data.get("name") == task_name:
            task_id_to_delete = task_id
            break

    if task_id_to_delete:
        return delete_task(task_id_to_delete)
    return False

def get_task(taskName):
    tasks = FBconn.get('/', 'Tasks') or {}
    for task_id, task_data in tasks.items():
        if isinstance(task_data, dict) and task_data.get("name") == taskName:
            return task_data
    return None

def get_all_tasks_dict():
    """Get all tasks as a dictionary with IDs as keys"""
    return FBconn.get('/', 'Tasks') or {}

def mark_task_complete(task_id):
    """Mark a task as complete"""
    tasks = FBconn.get('/', 'Tasks') or {}
    if task_id in tasks:
        task = tasks[task_id]
        task['completed'] = True
        FBconn.put('/Tasks', task_id, task)
        return True
    return False

def mark_task_complete_by_name(task_name):
    """Mark a task as complete by name - for backward compatibility"""
    tasks = FBconn.get('/', 'Tasks') or {}
    for task_id, task_data in tasks.items():
        if isinstance(task_data, dict) and task_data.get("name") == task_name:
            task_data['completed'] = True
            FBconn.put('/Tasks', task_id, task_data)
            return True
    return False

def get_user_by_username(user_name):
    users = FBconn.get('/', 'users') or {}
    user = users.get(user_name)
    return user

def insert_to_db_user(username, user_data):
    users = FBconn.get('/', 'users') or {}
    FBconn.put('/users', username, user_data)
    return True

def check_user_exists(username):
    users = FBconn.get('/', 'users') or {}
    return username in users

def get_coins_from_db(username):
    users = FBconn.get('/', 'users') or {}
    user = users.get(username)
    if user:
        return user.get('coins', 0)

def change_coins_for_user(username,coins):
    users = FBconn.get('/', 'users') or {}
    user = users.get(username)
    if user:
        currentCoins = user.get('coins', 0)
        FBconn.put('/users', username, {'coins': currentCoins + coins})
        return True
    return False

init_stats_if_needed()